In [1]:
!pip install weave

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ---------------------------------------- 0.0/765.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/765.9 kB ? eta -:--:--
   --------------------------- ------------ 524.3/765.9 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 765.9/765.9 kB 2.0 MB/s  0:00:00
   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   - -------------------------------------- 0.8/19.4 MB 4.2 MB/s eta 0:00:05
   --- ------------------------------------ 1.6/19.4 MB 4.7 MB/s eta 0:00:04
   ----- ---------------------------------- 2.6/19.4 MB 4.3 MB/s eta 0:00:04
   ------- -------------------------------- 3.7/19.4 MB 4.6 MB/s eta 0:00:04
   ---------- ----------------------------- 5.0/19.4 MB 5.0 MB/s eta 0:00:03
   ------------ --------------------------- 6.3/19.4 MB 5.3 MB/s eta 0:00:03

  DEPRECATION: Building 'intervaltree' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'intervaltree'. Discussion can be found at https://github.com/pypa/pip/issues/6334
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2025.11.3 requires xformers>=0.0.27.post2; ("linux" in sys_platform or sys_platform == "win32") and (platform_machine == "AMD64" or platform_machine == "x86_64"), which is not installed.
unsloth-zoo 2025.11.4 requires torchao>=0.13.0, which is not installed.


In [2]:
from datasets import load_dataset

data = load_dataset(
    "json",
    data_files={
        "train": "safelora_dpo_dataset.jsonl",
        "eval": "safelora_eval_dpo_120.jsonl",
    },
)

train_dataset = data["train"]
eval_dataset = data["eval"]

In [5]:
import weave
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from unsloth import FastLanguageModel, PatchDPOTrainer, is_bfloat16_supported
PatchDPOTrainer()

import torch
from trl import DPOTrainer, DPOConfig   

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
    device_map    = {"": 0}, 
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 64,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    max_seq_length = max_seq_length,
)

training_args = DPOConfig(
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 8,
    warmup_ratio = 0.1,
    num_train_epochs = 3,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    seed = 42,
    output_dir = "outputs",
    padding_value = tokenizer.pad_token_id, 
)

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = training_args,
    beta = 0.1,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
    max_length = 1024,
    max_prompt_length = 512,
)

dpo_trainer.train()


==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce GTX 1650 with Max-Q Design. Num GPUs = 1. Max memory: 4.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Unsloth 2025.11.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Extracting prompt in train dataset (num_proc=12):   0%|          | 0/5850 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=12):   0%|          | 0/5850 [00:00<?, ? examples/s]

RuntimeError: One of the subprocesses has abruptly died during map operation.To debug the error, disable multiprocessing.